# Data

kejahatan di wilayah metropolitan utama seperti London, terjadi dalam pola yang berbeda. data ini mencakup jumlah laporan kriminalitas menurut bulan, LSOA borough, kategori mayor/minor, dan tahun (2008 - 2016).

data df_LondonCrime berjumlah 13 juta data, yang berisi jumlah laporan kriminal menurut bulan, wilayah LSOA, dan kategori mayor/minor, dan tahun (2008 - 2016). jumlah data pada masing-masing tahun sebanyak 1.498.956 data, dengan 33 wilayah dan juga 9 kategori kejahatan mayor.

**variabel**\
lsoa_code: code for Lower Super Output Area in Greater London\
borough: Common name for London borough\
major_category: High level categorization of crime\
minor_category: Low level categorization of crime within major category\
value: monthly reported count of categorical crime in given borough\
year: Year of reported counts, 2008-2016\
month: Month of reported counts, 1-12

# Import Libraries

In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

# Read Dataset

In [20]:
#read csv
df_london = pd.read_csv('C:/Users/ASUS/H8_Python_KS09_MSIB _3/Assignment1/dataset/london_crime_by_lsoa.csv')
df_london

,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01001116,Croydon,Burglary,Burglary in Other Buildings,0,2016,11
1,E01001646,Greenwich,Violence Against the Person,Other violence,0,2016,11
2,E01000677,Bromley,Violence Against the Person,Other violence,0,2015,5
3,E01003774,Redbridge,Burglary,Burglary in Other Buildings,0,2016,3
4,E01004563,Wandsworth,Robbery,Personal Property,0,2008,6
...,...,...,...,...,...,...,...
13490599,E01000504,Brent,Criminal Damage,Criminal Damage To Dwelling,0,2015,2
13490600,E01002504,Hillingdon,Robbery,Personal Property,1,2015,6
13490601,E01004165,Sutton,Burglary,Burglary in a Dwelling,0,2011,2
13490602,E01001134,Croydon,Robbery,Business Property,0,2011,5


# Eksplaratory & Cleaning Data

In [21]:
#melihat dimensi data
df_london.shape

(13490604, 7)

dari output diatas, dapat dilihat bahwa jumlah data ada 13.490.604 data dengan 7 kolom. sehingga, akan dilanjutkan untuk mengecek missing value.

In [ ]:
#cek missing value
df_london.isnull().sum()

dari output diatas, dapat dilihat bahwa data tidak memiliki missing value. sehingga, akan dilanjutkan untuk melihat tipe data pada tiap kolom.

In [ ]:
#cek tipe data
df_london.info()

dari output diatas, dapat dilihat bahwa tipe data telah sesuai. sehingga akan dilanjutkan untuk mengubah nama kolom menjadi string.

In [ ]:
#ubah nama kolom jd string
df_london.columns = list(map(str, df_london.columns))

---
# A. Area Plot

**Question**

Bagaimana bentuk area plot untuk membandingkan 5 wilayah yang memiliki jumlah kriminalitas tertinggi di London tahun 2008 - 2016?

**Pre-processing**

In [ ]:
#menjumlahkan nilai pada kolom value sesuai dengan borough dan year
df_group = df_london.groupby(['borough', 'year'])[['value']].sum()

In [ ]:
#ubah nama kolom value jadi total
df_group.rename(columns={"value":"total"}, inplace=True)

In [ ]:
#reset index menjadi awal terlebih dahulu
df_group.reset_index(inplace=True)

In [ ]:
#menampilkan data
df_group

In [ ]:
#buat variabel years utk menampung list years yang ada di dalam dataframe df_group
boroughs = list(df_group["borough"].unique())

In [ ]:
#buat variabel years untuk menampung list years yang ada di dalam dataframe df_group
years = list(df_group["year"].unique())
years

In [ ]:
#inisiasi dataframe kosong dari kolom years dan index boroughs
df_boryear = pd.DataFrame(None, columns=years, index=boroughs)
df_boryear.head()

In [ ]:
#mengisi data pada dataframe df_boryear
for borough in boroughs:
    for year in years:
        df_boryear.at[borough, year] = int(df_group.loc[
            (df_group["borough"] == borough)
            & (df_group["year"] == year), "total"
        ])

In [ ]:
#buat kolom total yang akan digunakan  untuk mengurutkan borough
df_boryear["total"] = df_boryear.sum(axis=1)

In [ ]:
#urutkan data berdasarkan kolom total scr descending
df_boryear.sort_values(["total"], ascending=False, inplace=True)

In [ ]:
#ambil 5 data teratas
df_top5 = df_boryear.head()
df_top5

In [ ]:
#hapus kolom total karena tidak dibutuhkan dalam pembuatan area plot
del df_top5["total"]

In [ ]:
#transpose df_top5bor
df_top5bor = df_top5.transpose()
df_top5bor

**Visualisasi**

In [ ]:
#area plot
df_top5bor.index = df_top5bor.index.map(int)

df_top5bor.plot(kind='area',
            stacked=False,
            figsize=(20, 10),
            )

plt.title('5 Wilayah dengan Jumlah Kriminalitas Tertinggi di London Tahun 2008-2016')
plt.ylabel('Jumlah kriminalitas')
plt.xlabel('Tahun')

plt.show()

**Kesimpulan**

berdasarkan area plot diatas, dapat dilihat bahwa ada 5 wilayah dengan jumlah kriminalitas tertinggi di London tahun 2008 - 2016 yaitu Westminster, Lambeth, Southwark, Camden, dan Newham. wilayah Westminster mendominasi jumlah kriminalitas tertinggi di London tahun 2008 - 2016, adapun puncak paling tinggi kriminalitas di Westminster sebesar 59.226 ribu di tahun 2012.

---
# B. Histogram

**Question**

Berapa distribusi frekuensi yang memiliki jumlah kriminalitas tertinggi di London tahun 2013 jika disajikan dalam histogram? 

**Pre-processing**

In [ ]:
#pilih year 2013
y2013 = df_london[(df_london['year'] == 2013)]
y2013

In [ ]:
#urutkan data berdasarkan kolom value secara descending
y2013.sort_values(["value"], ascending=False, axis=0, inplace=True)

In [ ]:
#total kriminal tiap wilayah di tahun 2013
df_bor2013 = y2013.groupby(['borough']).sum()

In [ ]:
#hapus kolom yg tidak digunakan
df_bor2013.drop(['year','month'], axis=1, inplace=True)

In [ ]:
#urutkan data berdasarkan kolom total secara descending
df_bor2013.sort_values(['value'], ascending=False, axis=0, inplace=True)
df_bor2013.reset_index()

In [ ]:
#np.histogram return 2 values
count, bin_edges = np.histogram(df_bor2013)

#freq count
print(count)

#bin ranges, default=10 bins
print(bin_edges)   

**Visualisasi**

In [ ]:
#histogram
df_bor2013.plot(kind='hist', bins=14, figsize=(10, 6), color=['plum'])
plt.title('Jumlah Kriminalitas Tertinggi di London Tahun 2013')
plt.ylabel('Banyaknya Wilayah')
plt.xlabel('Jumlah Kriminalitas')
plt.show()

**interpretasi**

berdasarkan histogram diatas, dapat dilihat bahwa distribusi frekuensi jumlah kriminalitas tertinggi dari 33 wilayah di London tahun 2013. sehingga, didapat kisaran bin dan distribusi frekuensinya sebagai berikut:\
1 wilayah terjadi antara  110 - 5484.2 kriminalitas\
3 wilayah terjadi antara 5484.2 - 10858.4 kriminalitas\
5 wilayah terjadi antara 10858.4 - 16232.6 kriminalitas\
9 wilayah terjadi antara 16232.6 - 21606.8  kriminalitas\
9 wilayah terjadi antara 21606.8 - 26981 kriminalitas\
5 wilayah terjadi antara 26981 - 32355.2 kriminalitas\
1 wilayah terjadi antara 48477.8 - 538526 kriminalitas\

---
# C. Bar Chart


**Question**

Bagaimana bentuk visualisasi jumlah kriminalitas di wilayah Bromley tahun 2010 - 2016 menggunakan bar chart?

**Pre-processing**

In [ ]:
#pilih borough dan year
west = df_london[(df_london['borough'] == "Bromley") &
       (df_london['year'] >= 2010)]
west

In [ ]:
#urutkan data berdasarkan tahun
west.sort_values(['year','month','value'], ascending=True, axis=0, inplace=True)
west

In [ ]:
west.drop(['month'], axis=1, inplace=True)

In [ ]:
#sum year
df_year = west.groupby(['year']).sum()
df_year

**Visualisasi**

In [ ]:
df_year.plot(kind='bar', figsize=(8, 6), color='y')

plt.title('Jumlah Kriminalitas di Wilayah Bromley Tahun 2010 - 2016')
plt.ylim(ymin=0)
plt.xlabel('Tahun')
plt.ylabel('Jumlah Kriminalitas')

plt.show()

**Kesimpulan**

berdasarkan bar chart diatas, dapat dilihat bahwa jumlah kriminalitas tertinggi di wilayah Bromley sebesar 20.637 ribu di tahun 2015 sedangkan untuk jumlah kriminalitas terendahnya ada di tahun 2013 sebesar 18.945 ribu.

---
# D. Pie Chart

**Question**

Berapa jumlah proporsi (persentase) dari 5 wilayah yang memiliki kriminalitas tertinggi dalam kategori utama "Burglary" tahun 2016 dengan menggunakan pie chart?

**Pre-processing**

In [ ]:
#pilih major category dan year
bur = df_london[(df_london['major_category'] == "Burglary") &
               (df_london['year'] == 2016)]
bur

In [ ]:
#group
df_group = bur.groupby(['borough', 'year'])[['value']].sum()
df_group

In [ ]:
#ubah nama kolom value jadi total
df_group.rename(columns={"value":"total"}, inplace=True)

In [ ]:
#reset index menjadi awal terlebih dahulu
df_group.reset_index(inplace=True)

In [ ]:
df_group

In [ ]:
#buat variabel years utk menampung list years yg ada di dlm dataframe df_group
boroughs = list(df_group["borough"].unique())
boroughs

In [ ]:
#buat variabel years utk menampung list years yg ada di dlm dataframe df_group
years = list(df_group["year"].unique())
years

In [ ]:
#inisiasi dataframe kosong dari kolom years dan index boroughs
df_boryear = pd.DataFrame(df_group, columns=years, index=boroughs)
df_boryear.head()

In [ ]:
#mengisi data pada dataframe df_boryear
for borough in boroughs:
    for year in years:
        df_boryear.at[borough, year] = int(df_group.loc[
            (df_group["borough"] == borough)
            & (df_group["year"] == year), "total"
        ])

In [ ]:
#buat kolom total yg akan digunakan  utk mengurutkan borough
df_boryear["total"] = df_boryear.sum(axis=1)

In [ ]:
#urutkan data berdasarkan kolom total scr descending
df_boryear.sort_values(["total"], ascending=False, inplace=True)

In [ ]:
#cek borough sutton
df_top5 = df_boryear.head()
df_top5

In [ ]:
#reset index to put in back in as a column
df_top5.reset_index(inplace=True)

In [ ]:
df_top5

In [ ]:
#rename kolom
df_top5.rename(columns={'index':'borough'}, inplace=True)
df_top5.columns

In [ ]:
#set index borough
df_top5.set_index("borough")

In [ ]:
#group contries by borough and apply sum() function

df_borough = df_top5.groupby('borough', axis=0).sum()

print(type(df_top5.groupby('borough', axis=0)))

df_borough.head()

**Visualisasi**

In [ ]:
#plot data: dg passing kind='pie'

df_borough['total'].plot(kind='pie',
                            figsize=(5, 6),
                            autopct='%1.1f%%',
                            startangle=90,
                            shadow=True,
                           )

plt.title('Presentase Kriminalitas Tertinggi dari 5 Wilayah dalam Kategori Utama "Burglary" Tahun 2016')
plt.axis('equal')

plt.show()

**Kesimpulan**

berdasarkan pie chart diatas, dapat dilihat bahwa proporsi jumlah kriminalitas tertinggi dalam kategori utama "Burglary" tahun 2016 ada di wilayah Barnet sebesar 22%. sedangkan untuk jumlah kriminalitas terendahnya ada di wilayah Tower Hamlets sebesar 18.1%. sehingga, jika diurutkan dari proporsi jumlah kriminalitas tertinggi dalam kategori utama "Burglary" tahun 2016 yaitu wilayah Barnet, Westminster, Lambeth, Southwark, dan  Tower Hamlets.

---
# E. Box Plot

**Question**

Bagaimana visualisasi menggunakan box plot dalam jumlah kriminalitas di wilayah Sutton pada tahun 2008 - 2016?

**Pre-processing**

In [ ]:
#pilih borough
sutton = df_london[(df_london['borough'] == "Sutton")]
sutton

In [ ]:
#ubah nama kolom jd string
sutton.columns = list(map(str, sutton.columns))

In [ ]:
#group
df_group = sutton.groupby(['borough', 'year'])[['value']].sum()
df_group

In [ ]:
#ubah nama kolom value jadi total
df_group.rename(columns={"value":"total"}, inplace=True)

In [ ]:
#reset index menjadi awal terlebih dahulu
df_group.reset_index(inplace=True)

In [ ]:
df_group

In [ ]:
#buat variabel years utk menampung list years yg ada di dlm dataframe df_group
boroughs = list(df_group["borough"].unique())
boroughs

In [ ]:
#buat variabel years utk menampung list years yg ada di dlm dataframe df_group
years = list(df_group["year"].unique())
years

In [ ]:
#inisiasi dataframe kosong dari kolom years dan index boroughs
df_boryear = pd.DataFrame(df_group, columns=years, index=boroughs)
df_boryear.head()

In [ ]:
#mengisi data pada dataframe df_boryear
for borough in boroughs:
    for year in years:
        df_boryear.at[borough, year] = int(df_group.loc[
            (df_group["borough"] == borough)
            & (df_group["year"] == year), "total"
        ])

In [ ]:
#buat kolom total yg akan digunakan  utk mengurutkan borough
df_boryear["total"] = df_boryear.sum(axis=1)

In [ ]:
#urutkan data berdasarkan kolom total scr descending
df_boryear.sort_values(["total"], ascending=False, inplace=True)

In [ ]:
#cek borough sutton
df_top5 = df_boryear
df_top5

In [ ]:
#hapus kolom total krn tdk dibutuhkan dlm pembuatan box plot
del df_top5["total"]

In [ ]:
#transpose
df_top5bor = df_top5.transpose()

In [ ]:
df_top5bor.head()

In [ ]:
df_top5bor.describe()

**Visualisasi**

In [ ]:
df_top5bor.plot(kind='box', figsize=(8, 6), color='blue', vert=False)

plt.title('Box Plot Jumlah Kriminalitas di Wilayah Sutton Tahun 2008 - 2016')
plt.xlabel('Jumlah Kriminalitas')

plt.show()

**Kesimpulan**

berdasarkan box plot diatas, dapat dilihat bahwa data jumlah kriminalitas di wilayah Sutton tahun 2008 - 2016 tidak simetris karena data cenderung miring ke kanan (positive skewness) disertai whisker bagian atas (Q3) lebih panjang daripada bagian bawah (Q1) dan box plot ini tidak memiliki pencilan/outlier.

---
# F. Scatter Plot

**Question**

Bagaimana perubahan jumlah kriminalitas dalam kategori utama "Criminal Damage" di Wilayah Westminster tahun 2008 - 2016 dengan menggunakan scatter plot?

**Pre-processing**

In [ ]:
#pilih major category dan borough
df_crim = df_london[(df_london['borough'] == "Westminster") &
                   (df_london['major_category'] == "Criminal Damage")]
df_crim

In [ ]:
#group
df_group = df_crim.groupby(['borough', 'year'])[['value']].sum()

In [ ]:
#ubah nama kolom value jadi total
df_group.rename(columns={"value":"total"}, inplace=True)

In [ ]:
#reset index menjadi awal terlebih dahulu
df_group.reset_index(inplace=True)

In [ ]:
df_group

**Visualisasi**

In [ ]:
#scatter plot
df_group.plot(kind='scatter', x='year', y='total', figsize=(10, 6), color='blue')

plt.title('Jumlah Kriminalitas dalam Kategori Utama "Criminal Damage" di Wilayah Westminster Tahun 2008 - 2016')
plt.xlabel('Tahun')
plt.ylabel('Jumlah Kriminalitas')

plt.show()

**Kesimpulan**

berdasarkan scatter plot diatas, jumlah tingkat kriminalitas dalam kategori utama "Criminal Damage" di wilayah Westminster mengalami penurunan yang cukup signifikan di tahun 2008 - 2011. untuk jumlah tingkat kriminalitas tertinggi ada di tahun 2008 sebesar 2.599 ribu, sedangkan untuk terendahnya ada di tahun 2014 sebesar 2.016 ribu.

---
# F. Word Clouds

**Question**

Berapa jumlah wilayah terjadinya kriminalitas dalam kategori utama "Drugs" tahun 2009 dengan menggunakan word clouds?

**Pre-processing**

In [ ]:
#pilih major category dan year
drugs = df_london[(df_london['major_category'] == "Drugs") &
                  (df_london['year'] == 2009)]
drugs

In [ ]:
#group
df_group = drugs.groupby(['borough', 'year'])[['value']].sum()
df_group

In [ ]:
#ubah nama kolom value jadi total
df_group.rename(columns={"value":"total"}, inplace=True)

In [ ]:
#reset index menjadi awal terlebih dahulu
df_group.reset_index(inplace=True)

In [ ]:
df_group

In [ ]:
#buat variabel years utk menampung list years yg ada di dlm dataframe df_group
boroughs = list(df_group["borough"].unique())
boroughs

In [ ]:
#buat variabel years utk menampung list years yg ada di dlm dataframe df_group
years = list(df_group["year"].unique())
years

In [ ]:
#inisiasi dataframe kosong dari kolom years dan index boroughs
df_boryear = pd.DataFrame(df_group, columns=years, index=boroughs)
df_boryear.head()

In [ ]:
#mengisi data pada dataframe df_boryear
for borough in boroughs:
    for year in years:
        df_boryear.at[borough, year] = int(df_group.loc[
            (df_group["borough"] == borough)
            & (df_group["year"] == year), "total"
        ])

In [ ]:
#buat kolom total yg akan digunakan  utk mengurutkan borough
df_boryear["total"] = df_boryear.sum(axis=1)

In [ ]:
#urutkan data berdasarkan kolom total scr descending
df_boryear.sort_values(["total"], ascending=False, inplace=True)

In [ ]:
#cek data
df_boryear

In [ ]:
#total drugs thn 2009

total_drugs = df_boryear['total'].sum()
total_drugs

In [ ]:
max_words = 100
word_string = ''
for country in df_boryear.index.values:
    
    #chek if country's name is a single-word name
    if len(country.split(' ')) == 1:
        repeat_num_times = int(df_boryear.loc[country,'total']/float(total_drugs)*max_words)
        word_string = word_string + ((country + ' ') * repeat_num_times)
        
#display the generated text
word_string

In [ ]:
#create the word cloud
wordcloud = WordCloud(background_color='white').generate(word_string)

**Visualisasi**

In [ ]:
#word clouds
fig = plt.figure()
fig.set_figwidth(14)
fig.set_figheight(18)

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

**Kesimpulan**

berdasarkan word clouds diatas, jumlah kriminalitas dalam kategori utama "Drugs" tahun 2009 terdapat 24 wilayah yang ditampilkan oleh word clouds. wilayah Westminster dan Southwark menjadi jumlah kriminalitas terbanyak dalam kategori utama "Drugs" di tahun 2009, hal ini dilihat dari semakin besar dan tebal kata itu muncul pada word clouds.

# Overall Analysis

berdasarkan dari hasil visualisasi data sederhana yang telah dibuat menggunakan data kejahatan london (london crime data) pada tahun 2008-2016 terdapat sekitar 13 juta data dengan 33 wilayah dan 9 kategori kejahatan mayor/tingkat tinggi, sehingga dapat disimpulkan bahwa:
1. wilayah Weistminster memiliki jumlah aktivitas kriminal paling besar di London.
2. 5 wilayah yang paling banyak terjadi kriminalitas di London yaitu Westminster, Lambeth, Southwark, Camden dan Newham.
3. jumlah kriminalitas tertinggi dalam kategori utama "Burglary" tahun 2016 ada di wilayah Barnet sebesar 22%.
4. pada tahun 2009, wilayah Westminster dan Southwark menjadi jumlah kriminalitas tertinggi dalam kategori utama "Drugs"